<h1>About Dataset</h1>
<h2>Dataset Overview</h2>
<p>The <strong>Spending Patterns Dataset</strong> provides a synthetic representation of consumer spending behavior across various categories. This dataset is ideal for exploratory data analysis, statistical modeling, and machine learning applications related to financial forecasting, customer segmentation, or consumer behavior analysis.</p>
<h2>Dataset Features</h2>
<p>The dataset contains 10,000 transactions for 200 unique customers. Each transaction is associated with detailed information, including category, item, quantity, price, payment method, and transaction date.</p>
Data Source from Kaggle <br/>
<a href="https://www.kaggle.com/datasets/ahmedmohamed2003/spending-habits">Spending Habits</a>
https://www.kaggle.com/datasets/ahmedmohamed2003/spending-habits